In [ ]:
# -*- coding: utf-8 -*-

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2"

import tensorflow as tf
import numpy as np
import time

from plot_loader import plot_data


def initialize_session():
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 0.4
    return tf.Session(config=config)

train_acc_summary = [] # 학습용 데이터의 오류를 중간 중간 기록한다
test_acc_summary = []  # 테스트용 데이터의 오류를 중간 중간 기록한다



In [ ]:
##################################################
max_len = 800           # sequence 단어 수 제한
max_vocab = 50000       # maximum 단어 개수
BATCH_SIZE = 10         # 배치 사이즈
emb_dim = 256            # 단어 embedding dimension
hidden_dim = 256        # RNN hidden dim
learning_rate = 0.0005  # Learning rate
use_clip = True         # Gradient clipping 쓸지 여부
class_size = 4
train_acc_summary = [] # 학습용 데이터의 정확도를 중간 중간 기록한다
test_acc_summary = []  # 테스트용 데이터의 정확도를 중간 중간 기록한다
train_loss_summary = [] # 학습용 데이터의 정확도를 중간 중간 기록한다
test_loss_summary = []  # 테스트용 데이터의 정확도를 중간 중간 기록한다
##################################################
data = plot_data(max_vocab=max_vocab, max_len=max_len)
from genre_movie_rnn import Model
END_TOKEN = "<eos>"
model = Model(max_len=max_len,
              emb_dim=emb_dim,
              hidden_dim=hidden_dim,
              vocab_size=max_vocab,
              class_size=class_size,
              use_clip=True, learning_rate=learning_rate, end_token=data.w2idx[END_TOKEN])

sess = initialize_session()
sess.run(tf.global_variables_initializer())

In [ ]:
def test_model():
    total_batch = int(len(data.test_ids) / BATCH_SIZE)
    test_loss, test_acc = 0, 0
        
    for _ in range(total_batch):
        test_ids, length, label = data.get_test(BATCH_SIZE)
        loss, acc = sess.run([model.loss, model.accuracy], feed_dict={model.x: test_ids, model.x_len: length, model.y: label})

        test_loss += loss
        test_acc += acc
        
    return test_loss/total_batch, test_acc/ total_batch
#     print(" --> test_loss: {:.3f} | test_acc: {:.3f}".format(test_loss / test_cnt, same/test_cnt/BATCH_SIZE))

total_batch = int(len(data.train_ids) / BATCH_SIZE)
for epoch in range(1):
    
    train_avg_loss, train_avg_acc = 0, 0
    
    for it in range(total_batch):
        train_ids, length, label = data.get_train(BATCH_SIZE)
        
        loss, acc, _, xx = sess.run([model.loss, model.accuracy, model.update, model.x_emb],
                            feed_dict={model.x: train_ids, model.x_len: length, model.y: label, model.keep_prob: 0.75})
        print(xx.shape)
        train_avg_loss += loss / total_batch
        train_avg_acc += acc / total_batch

    test_avg_loss, test_avg_acc = test_model()
    
    train_acc_summary.append(train_avg_acc)
    test_acc_summary.append(test_avg_acc)
    train_loss_summary.append(train_avg_loss)
    test_loss_summary.append(test_avg_loss)
    
    print("epoch {} - train_loss: {:.4f}, train_acc: {:.4f}, test_loss: {:.4f}, test_acc: {:.4f}"
          .format(epoch+1, train_avg_loss, train_avg_acc, test_avg_loss, test_avg_acc))  
    


In [1]:
a = [1, 2, 3, 4]

In [3]:
import tensorflow as tf
b = tf.reshape(a, (10, 4))

/etc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


ValueError: Cannot reshape a tensor with 4 elements to shape [10,4] (40 elements) for 'Reshape' (op: 'Reshape') with input shapes: [4], [2] and with input tensors computed as partial shapes: input[1] = [10,4].